In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile
!pip install torch-points3d

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

!python train.py -s /content/gaussian-splatting/tandt/train

In [ ]:
from utils.read_write_model import read_model

cam_intrinsics, images_metas, points3d = read_model(path)

pts_indices = np.array([points3d[key].id for key in points3d])
pts_xyzs = np.array([points3d[key].xyz for key in points3d])

In [ ]:
from torch_points3d.applications.pointnet2 import PointNet2
import torch

pn_model = PointNet2(
    "segmentation",
    input_nc=3,  # Number of input features 
    num_classes=16,  # Number of segmentation classes 
    pretrained=True,
)

pn_model.eval()

In [ ]:
with torch.no_grad():
    outputs = pn_model(point_cloud)  # Shape: (batch_size, num_points, num_classes)
for classes in outputs[::3]:
    #Save Individual Segments to be viewed
    #highlight in original image


predicted_labels = torch.argmax(outputs, dim=-1)  # Shape: (batch_size, num_points)

In [ ]:
from scene.dataset_readers import storePly
storePly(path, xyz, rgb)